In [1]:
import pandas as pd
import numpy as np

from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

Batch

In [2]:
df_all = pd.read_csv('./input/RNASeq_counts/FPKM.all.array.txt', sep='\t', index_col=0)

In [3]:
path = '../../SCProjects/genome/Scerevisiae_S288C/GCA_000146045.2_R64_rna_from_genomic.fna'
x = SeqIO.parse(path, format='fasta')

rna_seqlib = dict()
for i in x:
    ltag = [j for j in i.description.split(' ') if 'locus_tag=' in j][0]
    ltag = ltag.split('=')[-1][:-1]
    rna_seqlib[ltag] = str(i.seq.transcribe())
    
path = './input/RNASeq_counts/added_gene_CDS.fsa'
x = SeqIO.parse(path, format='fasta')
for i in x:
    ltag = i.description.split(' ')[0]
    ltag = ltag.split('_')[0]
    rna_seqlib[ltag] = str(i.seq.transcribe())

In [4]:
df_avg = pd.DataFrame(columns=['A', 'U', 'G', 'C'],
                      index=['Array.b', 'Gsnap.b', 'Stampy.b', 'Tophat.b', 'RNASeq.b', 'Avg.b',
                             'Array.c', 'Gsnap.c', 'Stampy.c', 'Tophat.c', 'RNASeq.c', 'Avg.c',
                             'Avg'])

cols_map = {'Array.b': ['A.b1', 'A.b2', 'A.b3'],
            'Gsnap.b': ['G.b1', 'G.b2', 'G.b3'],
            'Stampy.b': ['S.b1', 'S.b2', 'S.b3'],
            'Tophat.b': ['T.b1', 'T.b2', 'T.b3'],
            'Array.c': ['A.c1', 'A.c2', 'A.c3'],
            'Gsnap.c': ['G.c1', 'G.c2', 'G.c3'],
            'Stampy.c': ['S.c1', 'S.c2', 'S.c3'],
            'Tophat.c': ['T.c1', 'T.c2', 'T.c3']}

bs = ['A', 'U', 'G', 'C']

for method,cols in cols_map.items():
    counts = {col:{b:0 for b in bs} for col in cols}
    comp = {col:{b:0 for b in bs} for col in cols}
    for col in cols:
        for g in df_all.index:
            for b in bs:
                counts[col][b] += df_all.loc[g,col] * rna_seqlib[g].count(b)

        tot = sum(counts[col].values())
        for b in bs:
            comp[col][b] = counts[col][b] / tot
            
    for b in bs:
        df_avg.loc[method,b] = np.mean([i[b] for i in comp.values()])

In [5]:
for suffix in ['b', 'c']:
    cols = [col + '.' + suffix for col in ['Gsnap', 'Stampy', 'Tophat']]
    df_avg.loc['RNASeq.' + suffix, :] = np.mean(df_avg.loc[cols, :], axis=0)
    cols = [col + '.' + suffix for col in ['Array', 'RNASeq']]
    df_avg.loc['Avg.' + suffix, :] = np.mean(df_avg.loc[cols, :], axis=0)
    
df_avg.loc['Avg'] = (df_avg.loc['Avg.b'] + df_avg.loc['Avg.c']) / 2

In [6]:
df_avg.loc[['Avg.b', 'Avg.c', 'Avg']]

,A,U,G,C
Avg.b,0.310292,0.278057,0.214037,0.197614
Avg.c,0.310517,0.276804,0.214006,0.198672
Avg,0.310404,0.27743,0.214022,0.198143


In [16]:
for b in bs:
    print(b, round(df_avg.loc['Avg',b], 6))

A 0.310404
U 0.27743
G 0.214022
C 0.198143


In [14]:
sum(df_avg.loc['Avg',:])

1.0

In [43]:
df_avg.to_excel('./input/RNASeq_counts/read_counts.xlsx', index=True)